In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd


df = pd.read_csv("old-data-jul-9.csv")

df = df.drop("Winner 2", axis=1)


print(df.columns.tolist())

df.head()


['Team Start', 'Team 1 Player 1 ', 'Team 1 Player 2 ', 'Team 2 Player 1 ', 'Team 2 Player 2 ', 'Team 1 Score ', 'Team 2 Score', 'Winning Team']


,Team Start,Team 1 Player 1,Team 1 Player 2,Team 2 Player 1,Team 2 Player 2,Team 1 Score,Team 2 Score,Winning Team
0,Team 1,Erick A,Miles W,Mathis W,Nico U,19,17,Team 1
1,Team 2,Angel C,Anthony C,Erick A,Miles W,12,15,Team 2
2,Team 1,Erick A,Miles W,Mathis W,Nico U,17,15,Team 1
3,Team 1,Angel C,Miles W,Anthony C,Nico U,9,15,Team 2
4,Team 1,Angel C,Anthony C,Mathis W,Nico U,11,15,Team 2


In [3]:
players = pd.unique(df[['Team 1 Player 1 ', 'Team 1 Player 2 ', 'Team 2 Player 1 ', 'Team 2 Player 2 ']].values.ravel())

players

array(['Erick A', 'Miles W', 'Mathis W', 'Nico U ', 'Angel C',
       'Anthony C', 'Nico U'], dtype=object)

In [4]:
elo_scores: dict[str, float] = {player: 1000 for player in players}
elo_scores

{'Erick A': 1000,
 'Miles W': 1000,
 'Mathis W': 1000,
 'Nico U ': 1000,
 'Angel C': 1000,
 'Anthony C': 1000,
 'Nico U': 1000}

In [ ]:
# Implement basic elo. 
# Increase both players on each team by the same amount

def expected_score_for_r1(R1, R2):
    exp = (R2 - R1)/400
    return 1/(1 + 10**exp)

# K set to 32 like chess
def calc_new_rating(old_rating, actual, expected):
    return old_rating + (32 * (actual - expected))

team1_columns = ['Team 1 Player 1 ', 'Team 1 Player 2 ']
team2_columns = ['Team 2 Player 1 ', 'Team 2 Player 2 ']

# For game in games
for row in df.to_dict(orient="records"):
    winning_team = row["Winning Team"]

    if winning_team == "Team 1":
        winning_players = [row[col] for col in team1_columns]
        losing_players = [row[col] for col in team2_columns]
    
    else:
        winning_players = [row[col] for col in team2_columns]
        losing_players = [row[col] for col in team1_columns]
    
    print("Winning players:", winning_players)
    print("Losing players:", losing_players)

    # Get averaged team elo
    winning_elo= (sum(elo_scores[player] for player in winning_players if player in elo_scores) / 2)
    losing_elo = (sum(elo_scores[player] for player in losing_players if player in elo_scores) / 2)

    # Total % adds to 1
    winning_expected = expected_score_for_r1(winning_elo, losing_elo)
    losing_expected = 1 - winning_expected

    # 1 is a win, 0 is a loss
    new_winning_elo = calc_new_rating(winning_elo, 1, winning_expected)
    new_losing_elo = calc_new_rating(losing_elo, 0, losing_expected)

    for player in winning_players:
        elo_scores[player] = new_winning_elo
        print(f"Updated ELO for {player}: {elo_scores[player]:.1f}")

    for player in losing_players:
        elo_scores[player] = new_losing_elo
        print(f"Updated ELO for {player}: {elo_scores[player]:.1f}")


Winning players: ['Erick A', 'Miles W']
Losing players: ['Mathis W', 'Nico U ']
Updated ELO for Erick A: 1035.9
Updated ELO for Miles W: 1035.9
Updated ELO for Mathis W: 973.3
Updated ELO for Nico U : 973.3
Winning players: ['Erick A', 'Miles W']
Losing players: ['Angel C', 'Anthony C']
Updated ELO for Erick A: 1049.7
Updated ELO for Miles W: 1049.7
Updated ELO for Angel C: 972.4
Updated ELO for Anthony C: 972.4
Winning players: ['Erick A', 'Miles W']
Losing players: ['Mathis W', 'Nico U ']
Updated ELO for Erick A: 1062.2
Updated ELO for Miles W: 1062.2
Updated ELO for Mathis W: 960.7
Updated ELO for Nico U : 960.7
Winning players: ['Anthony C', 'Nico U']
Losing players: ['Angel C', 'Miles W']
Updated ELO for Anthony C: 1008.1
Updated ELO for Nico U: 1008.1
Updated ELO for Angel C: 1000.1
Updated ELO for Miles W: 1000.1
Winning players: ['Mathis W', 'Nico U ']
Losing players: ['Angel C', 'Anthony C']
Updated ELO for Mathis W: 978.7
Updated ELO for Nico U : 978.7
Updated ELO for Angel C

In [ ]:
print(f"Total games played: {len(df)}\n")
print("Final Scores:")
for player, score in sorted(elo_scores.items(), key=lambda x: x[1], reverse=True):
    print(f"{player.strip():<15} {round(score, 1)}")


Total games played: 9

Final Score:
Erick A         1062.2
Nico U          1008.1
Miles W         1000.1
Angel C         984.7
Anthony C       984.7
Mathis W        980.1
Nico U          980.1
